In [1]:
from Bio import Seq
from collections import defaultdict
import pandas as pd
import numpy as np
import pathlib
from sklearn.ensemble import RandomForestClassifier

FRAME_LENGTH = 1500
K_MER = 4

In [2]:
def generate_words(chars, length):
    if length == 0:
        yield ''
    else:
        for c in chars:
            for suffix in generate_words(chars, length - 1):
                yield c + suffix
                
def get_features():
    nucleotides = generate_words('ACGT', K_MER)
    seen = set()
    features = []
    nucleotide2feature = dict()
    for n in nucleotides:
        if n in seen or Seq.reverse_complement(n) in seen:
            continue
        nucleotide2feature[n] = n 
        nucleotide2feature[Seq.reverse_complement(n)] = n
        seen.add(n)
        features.append(n)
    return features, nucleotide2feature
        
features, mapping = get_features()

In [3]:
def read_training_data(filename):
    data = []
    with open(filename, 'r') as f:
        lines = f.readlines()
        for line in lines:
            if line.startswith('>'):
                continue
            data.append(line.upper().strip())
    return data

positive_data = read_training_data('./data/vista1500')
negative_data = read_training_data('./data/randoms1500')

In [4]:
def get_feature_from_frame(frame):
    dd = dict()
    for feature in features:
        dd[feature] = 0
    total = len(frame) - K_MER
    for i in range(total):
        dd[mapping[frame[i:i+K_MER]]] += 1 / total
    return dd
    
def convert_data_to_features(data, label):
    data_obj = []
    for line in data:
        feature_obj = get_feature_from_frame(line)
        feature_obj['label'] = label
        data_obj.append(feature_obj)
    return data_obj

training_data = convert_data_to_features(positive_data, 0) + convert_data_to_features(negative_data, 1) 

In [5]:
training_df = pd.DataFrame(training_data)
training_df

,AAAA,AAAC,AAAG,AAAT,AACA,AACC,AACG,AACT,AAGA,AAGC,...,TACA,TAGA,TATA,TCAA,TCCA,TCGA,TGAA,TGCA,TTAA,label
0,0.016043,0.007353,0.014706,0.012032,0.008021,0.007353,0.003342,0.004679,0.014706,0.008690,...,0.006016,0.008021,0.000668,0.008021,0.007353,0.000668,0.009358,0.004679,0.004679,0
1,0.007353,0.006016,0.006016,0.006684,0.007353,0.009358,0.002005,0.002005,0.008021,0.004011,...,0.006016,0.002005,0.000000,0.001337,0.010695,0.000000,0.004679,0.007353,0.002674,0
2,0.028743,0.012701,0.017380,0.022059,0.011364,0.007353,0.003342,0.010027,0.014706,0.010695,...,0.014706,0.004679,0.003342,0.008021,0.008021,0.000668,0.014706,0.004011,0.009358,0
3,0.010695,0.007353,0.014706,0.019385,0.009358,0.007353,0.000000,0.010695,0.012032,0.004011,...,0.008690,0.008021,0.001337,0.010695,0.011364,0.000668,0.012032,0.008021,0.006684,0
4,0.018717,0.012032,0.022059,0.021390,0.016043,0.006684,0.004679,0.006684,0.016043,0.009358,...,0.009358,0.007353,0.004679,0.012032,0.012701,0.000668,0.010695,0.009358,0.010027,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2330,0.034759,0.015374,0.014037,0.023396,0.009358,0.010027,0.001337,0.012032,0.019385,0.007353,...,0.010695,0.007353,0.008690,0.010027,0.012032,0.000000,0.015374,0.000668,0.007353,1
2331,0.052807,0.011364,0.010027,0.030080,0.010695,0.006684,0.000000,0.011364,0.006684,0.004679,...,0.012032,0.010027,0.002674,0.012701,0.003342,0.000000,0.016711,0.004011,0.009358,1
2332,0.025401,0.015374,0.009358,0.016711,0.011364,0.008021,0.000668,0.012032,0.008021,0.003342,...,0.026070,0.008021,0.042112,0.008021,0.008021,0.000000,0.012032,0.004679,0.004679,1
2333,0.028075,0.014706,0.012032,0.021390,0.016043,0.005348,0.000668,0.011364,0.014706,0.006016,...,0.008690,0.002005,0.004011,0.013369,0.004679,0.000668,0.012701,0.006016,0.008021,1


In [6]:
clf = RandomForestClassifier()
clf.fit(training_df.drop(columns=['label']), training_df['label'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [7]:
def get_chromosome_frames(filename = './data/chr21.fa'):
    frame = ''
    with open(filename, 'r') as f:
        f.readline() # consume '>chr21'
        for line in f:
            frame += line.upper().strip()
            while len(frame) >= FRAME_LENGTH:
                yield frame[:FRAME_LENGTH]
                frame = frame[FRAME_LENGTH // 2:]
    

In [8]:
def get_chromosome_features():
    for frame in get_chromosome_frames():
        if 'N' in frame:
            continue
        yield get_feature_from_frame(frame)



In [9]:
computed_features = list(get_chromosome_features())
predictions = clf.predict_proba(pd.DataFrame(computed_features))
average_score = np.average(predictions,axis=0)

In [10]:
def save_results(filename, predictions):
    step = FRAME_LENGTH // 2
    filepath = pathlib.Path(filename).expanduser()
    filepath.parents[0].mkdir(parents=True, exist_ok=True)
    i = 0
    predictions_index = 0
    with open(str(filepath), 'w') as f:
        f.write('track type=wiggle_0 name="nowikowski" description="enhancers prediction" visibility=full autoScale=off vieLimits=0.0:1.0 color=50,150,255 yLineMark=11.76 yLineOnOff=on priority=10\n')
        f.write(f'variableStep chrom=chr21 span=1500\n')
        for frame in get_chromosome_frames():
            score = average_score[0]
            if 'N'not in frame:
                score = predictions[predictions_index][0]
                predictions_index += 1
            f.write(f'{i * step + 1} {score}\n')
            i += 1


In [11]:
save_results('./output/result.wig', predictions)